In [1]:
# enable reloading
%load_ext autoreload
# all the modules should be reloaded before executing the code
%autoreload 2

In [ ]:
import asyncio
from pathlib import Path

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage, filter_messages
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, StateGraph
from langgraph.types import Command

In [ ]:
"""Research Agent Subgraph."""
try:
    from .configuration import Configuration
    from .prompts import COMPRESS_RESEARCH_SIMPLE_HUMAN_MESSAGE, COMPRESS_RESEARCH_SYSTEM_PROMPT
    from .states import ResearcherOutputState, ResearchState
    from .utils import execute_tool_safely, get_all_tools, openai_websearch_called
except ImportError:
    import rootutils

    rootutils.setup_root(search_from=str(Path.cwd().parent), indicator=[".git", "pyproject.toml"], pythonpath=True)
    from src.agent.configuration import Configuration
    from src.agent.prompts import COMPRESS_RESEARCH_SIMPLE_HUMAN_MESSAGE, COMPRESS_RESEARCH_SYSTEM_PROMPT
    from src.agent.states import ResearcherOutputState, ResearchState
    from src.agent.utils import execute_tool_safely, get_all_tools, openai_websearch_called

In [ ]:
# Initialize a configurable model that we will use throughout the agent
configurable_model = init_chat_model(
    configurable_fields=("model", "max_tokens", "api_key"),
)


async def research_agent(state: ResearchState, config: RunnableConfig):
    config = Configuration.from_runnable_config(config)
    research_msgs = state.get("research_messages", [])
    tools = await get_all_tools(config)

    if len(tools) <= 1:  # ResearchComplete is default tool in the list
        msg = "No tools found to conduct research, please configure Search API"
        raise ValueError(msg)

    research_model_config = {
        "model": config.research_model,
        "max_tokens": config.research_model_max_tokens,
        # "api_key": config.research_model_api_key,
        "tags": ["langsmith:nostream"],
    }

    research_model = (
        configurable_model.bind_tools(
            tools,
        )
        .with_retry(stop_after_attempt=config.max_structured_output_retries)
        .with_config(research_model_config)
    )

    response = await research_model.ainvoke(
        research_msgs,
    )

    return Command(
        goto="research_tools",
        update={
            "research_messages": [response],
            "tool_call_iterations": state.get("tool_call_iterations", 0) + 1,
        },
    )


async def research_tools(state: ResearchState, config: RunnableConfig):
    config = Configuration.from_runnable_config(config)
    research_msgs = state.get("research_messages", [])
    most_recent_message = research_msgs[-1]

    # Early exit Criteria: No tools calls (or native web search calls) were made by the researcher
    if not most_recent_message.tool_calls and not openai_websearch_called(most_recent_message):
        return Command(goto="compress_research")

    # Otherwise, execute tools and gather results
    tools = await get_all_tools(config)

    tools_by_name = {tool.name if hasattr(tool, "name") else tool.get("name", "web_search"): tool for tool in tools}

    tool_calls = most_recent_message.tool_calls

    async_responses = [
        execute_tool_safely(tools_by_name[tool_call["name"]], tool_call["args"], config) for tool_call in tool_calls
    ]
    observations = await asyncio.gather(*async_responses)
    tool_outputs = [
        ToolMessage(
            content=observation,
            name=tool_call["name"],
            tool_call_id=tool_call["id"],
        )
        for observation, tool_call in zip(observations, tool_calls, strict=False)
    ]
    # Late Exit Criteria: We have exceeded our max guardrail tool call iterations or the most recent message contains ResearchComplete tool call
    # These are late exit criteria because we need to add ToolMessage
    update = {"research_messages": tool_outputs}
    if state.get("tool_call_iterations", 0) >= config.max_react_tool_calls or any(
        tool_call["name"] == "ResearchComplete" for tool_call in tool_calls
    ):
        return Command(
            goto="compress_research",
            update=update,
        )
    return Command(goto="research_agent", update=update)


async def compress_research(state: ResearchState, config: RunnableConfig):
    config = Configuration.from_runnable_config(config)
    synthesize_attempts = 0
    compression_model = configurable_model.with_config(
        {
            "model": config.compression_model,
            "max_tokens": config.compression_model_max_tokens,
            # "api_key": config.compress_model_api_key,
        },
    )
    researcher_msgs = state.get("research_messages", [])

    # Update the system prompts to now focus on compression rather than research
    researcher_msgs[0] = SystemMessage(
        content=COMPRESS_RESEARCH_SYSTEM_PROMPT,
    )
    researcher_msgs.append(HumanMessage(content=COMPRESS_RESEARCH_SIMPLE_HUMAN_MESSAGE))
    while synthesize_attempts < config.compression_attempts:
        try:
            response = await compression_model.ainvoke(
                researcher_msgs,
            )
            return {
                "compressed_research": str(response.content),
                "raw_notes": [
                    "\n".join(
                        [str(m.content) for m in filter_messages(researcher_msgs, include_types=["tool", "ai"])],
                    ),
                ],
            }
        except Exception as e:
            # import traceback

            # print(e, traceback.format_exc())
            synthesize_attempts += 1
            # TODO(@viv): #123 Add token limit check
            # if is_token_limit_exceeded(e, configurable.research_model):
            #     researcher_msgs = remove_up_to_last_ai_message(researcher_msgs)
            #     print(f"Token limit exceeded while synthesizing: {e}. Pruning the messages to try again.")
            #     continue
            print(f"Error synthesizing research report: {e}")
    return {
        "compressed_research": "Error synthesizing research report: Maximum retries exceeded",
        "raw_notes": [
            "\n".join([str(m.content) for m in filter_messages(researcher_msgs, include_types=["tool", "ai"])]),
        ],
    }


research_builder = StateGraph(ResearchState, output_schema=ResearcherOutputState, config_schema=Configuration)
research_builder.add_node("research_agent", research_agent)
research_builder.add_node("compress_research", compress_research)
research_builder.add_node("research_tools", research_tools)

research_builder.add_edge(START, "research_agent")
research_builder.add_edge("compress_research", END)
researcher_graph = research_builder.compile(name="Research Agent")

In [ ]:
conversation = [
    #  HumanMessage(content=PROJECT_IDEA),
    AIMessage(""" 1. **Target Audience and Purpose:**
   - You mentioned this is a personal project for fun and learning, as well as to showcase your skills to potential interviewers. Is there a specific aspect of the project you want to highlight to interviewers (e.g., AI capabilities, UI design, integration skills)?
2. **Image Processing:**
   - For the feature that takes pictures of handwritten notes, do you have any specific requirements or preferences for the image capture process (e.g., camera integration, file upload)?
   - Are there any particular challenges you anticipate with processing equations and block diagrams, and do you have any existing tools or libraries in mind to handle these?
3. **Integration with Notion:**
   - Could you elaborate on how you envision the integration with Notion? For example, do you have a specific structure or organization in mind for the notes within Notion?
   - Are there any specific Notion API functionalities you plan to use or any constraints you are aware of?
4. **LangGraph and UI:**
   - You mentioned using LangGraph's prebuilt UI. Could you provide more details on how you plan to utilize it? Are there any customizations or additional features you want to add to the UI?
5. **Multi-Agent System:**
   - You are considering a multi-agent system. Could you describe how you envision the interaction between the agents, and what specific roles each agent will play?
6. **MVP and Timeline:**
   - For the MVP, you aim to convert images to text and format them properly within two weeks. Are there any specific milestones or deliverables you have in mind for this initial phase?
7. **Future Development:**
   - You mentioned the possibility of developing a simple Android app using Flutter later on. Do you have any specific features or functionalities in mind for the mobile version?
 """),
    HumanMessage(
        content="""
1. nothing specific, but good working app, because i am going to use it regularly
2. for the time being we can go with file upload and when i decide to make android app using flutter then we can integrate images directly from the camera
3. I am thinking to fetch all the pages and sub pages and find pages with name and create a sub page and add notes over there. for example if topic is about optimizer then i will find page with deep learning and create a new sub page and add notes over there.
4. No, first i want to use only langgraph UI, it is really nice and easy to use
5. I am also thinking to use multi agent system one agent for image to text conversion and 2nd agent for text to notion or formatting (markdown)
6. no nothing specific milestone or anything
7. no just simple camera app with little instruction/chat box for Agent if needed
""",
    ),
    AIMessage("""1. **App Usage and Quality:**
   - You want a reliable and well-functioning app since you'll be using it regularly. Are there any specific performance or usability criteria you have in mind to ensure it meets your needs?
2. **Image Capture and Processing:**
   - For now, you prefer to start with a file upload feature for images. When you transition to an Android app using Flutter, do you have any specific requirements for how the camera integration should work?
3. **Notion Integration:**
   - You plan to fetch pages and sub-pages in Notion, find specific pages by name, and create sub-pages to add notes. Could you clarify if you have a specific naming convention or criteria for identifying these pages?
   - How do you envision the notes being structured or formatted once they are added to Notion?
4. **LangGraph UI:**
   - You are satisfied with using LangGraph's UI as it is. Are there any specific features or interactions within the UI that are crucial for your workflow?
5. **Multi-Agent System:**
   - You plan to use one agent for image-to-text conversion and another for text formatting and integration with Notion. Could you describe any specific interactions or data flow between these agents?
6. **Milestones and Timeline:**
   - While you don't have specific milestones, do you have a rough timeline or order of priorities for implementing different features?
7. **Future Android App:**
   - For the future Android app, you mentioned a simple camera app with a chat box for agent instructions. Are there any additional features or user interactions you envision for this app?
Please let me know if there are any other aspects of the project you'd like to discuss or clarify further.
 """),
    HumanMessage(
        content="1. no, i have not thought about it yet, we can check after first MVP is done"
        "2.i will use langgraph UI to upload the images for the time being"
        "3. simple chat ui with upload image button and text input box"
        "4. No not yet, because i have not use it. i will see once we reach to that point"
        "5. first agent will convert image to text and send this text information to 2nd agent and 2nd agent will format it and send it to notion"
        "6. no milestone or anything"
        "7.no nothing",
    ),
]
PROJECT_IDEA = (
    "create plan to develop Agentic AI note taking app using langgraph for my personal use (personal project for fun and learning) and i also want to show off my skills to my potential interviewer to get hired. it should do following "
    "1. take pictures of hand-written notes "
    "2. it will automatically format the hand-written notes (it might contains equations and block diagrams) "
    "3. find proper section (if section found then create sub page or create a new page) in my notion "
    "4. add this notes with proper format"
    "5. i want to use LangGraph's pre-build UI for interaction from PC"
    "6. for MVP (which can convert image to text and format it properly) in 2 weeks"
    "7. I am also thinking to use multi agent system one agent for image to text conversion and 2nd agent for text to notion or formatting (markdown)"
)

In [ ]:
# async for event in researcher_graph.astream(
#     {
#         "research_messages": [*conversation, HumanMessage(content=research_brief)],
#         #   "research_brief": research_brief,
#         "research_topic": PROJECT_IDEA,
#     },
#     stream_mode="update",
# ):
#     print(event)